In [1]:
import pandas as pd
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [2]:
def rmEmoji(data):
    return data.encode('euc-kr', 'ignore').decode('euc-kr')

# 한옥 데이터 전처리

In [3]:
hanok = pd.read_csv('CSV/뷰펜션.csv')

In [4]:
sukso_recommendation = pd.read_csv('CSV/숙소추천.csv')
sukso = pd.read_csv('CSV/숙소.csv')

In [5]:
# hanok = hanok.drop('Unnamed: 0', axis = 1)

# hanok_hash 전처리

In [6]:
hanok_hash_list = [] # 비어있는 리스트 만들기 
for txt in hanok['hashtag'] : 
    del_br = txt.strip("[]")
    del_br = del_br.replace("#","")
    del_br = del_br.replace(",","")
    del_br = del_br.replace("'","")
    
    hanok_hash_list.append(del_br) #값 채우기

In [7]:
hanok_hash_series = pd.Series(hanok_hash_list)

In [8]:
hanok_tf = hanok_hash_series.str.contains("숙소") == 0
hanok_tf = hanok_hash_series.str.contains("숙소추천") == 0
hanok_hash_series = hanok_hash_series[hanok_tf]

In [9]:
hanok_hash_series.reset_index(drop = True, inplace = True)

# hanok_df 전처리

In [10]:
hanok_df = hanok[hanok['hashtag'].str.contains("숙소") ==0]
hanok_df = hanok[hanok['hashtag'].str.contains("숙소추천") ==0]

In [11]:
index_del = hanok_df[hanok_df['hashtag'] == '[]'].index
hanok_df = hanok_df.drop(index_del)

In [12]:
hanok_df.reset_index(drop = True, inplace = True)

In [13]:
hanok_df['hashtag'] = hanok_hash_series

In [14]:
for i in range(len(hanok_df['hashtag'])):
    if(len(hanok_df['hashtag'].iloc[i])<=2):
        hanok_df['hashtag'].iloc[i] = rmEmoji(hanok_df['hashtag'].iloc[i])

C:\Users\morni\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [15]:
index_del = hanok_df[hanok_df['hashtag'] == ''].index
hanok_df = hanok_df.drop(index_del)

In [16]:
hanok_df.reset_index(drop = True, inplace = True)

In [17]:
hanok_df

,id,date,like,text,hashtag,src,target
0,sss.__.ong,2020-11-23 00:00:00+00:00,256,굿모닝🔆 31주,트래블리맵,NaN,1
1,kkwak_chaeum,2020-10-18 00:00:00+00:00,50,🤍👰🏻🤵🏻🤍 예정되어있던 결혼식날에 둘이서 ...,레터링풍선 웨딩풍선 사천여행 삼첨포여행 사천펜션 사천로뎀펜션 로뎀펜션 뷰펜션 삼천포...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,1
2,hey.ririi_,2020-10-13 00:00:00+00:00,192,(2) 바로 앞에 바닷가 있는 뷰 좋은 펜션...,지훈❤️혜리 포항여행 포항펜션 포항커플ᄑ...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,1
3,ddggungs,2019-09-10 00:00:00+00:00,41,- - 촌놈드을🤣 수영복입고 하쟸더니 그세를 못참고ㅎㅎ - 입주할집에가능하다면 테라...,지난주말 태풍온날 링링태풍뚫고 먼저도착해서여유부리기 경남하동펜션 뷰펜션 스파펜션 인...,NaN,1
4,jennifer_the_pursuer,2020-12-28 00:00:00+00:00,91,ㅡthis is heaven like that. ㅡ 길냥이들이 비싼 프렌치렉을 우리...,나만알고싶은곳 엄마랑 조카랑 힐링여행 대박 스파펜션 바베큐타임 양고기 프렌치렉 뷰펜...,NaN,1
...,...,...,...,...,...,...,...
938,kjy._.yony,2016-06-06 00:00:00+00:00,32,. 너무좋다 다시오길 너무 잘했다❤ . 264주,여름휴가 방포해수욕장 뷰펜션 바베큐 물놀이 튜브 겨들랑이이프다그램 소주 린 맥주 삼...,NaN,1
939,heehee_ao,2015-12-04 00:00:00+00:00,28,송년회.....하다가 얼어쥭겠당😩 . 290주,삼겹살 카프리 보드카 피치 극한직업 남편 자이글 고기 굽어 저녁식사 꿀맛 뷰펜션 소풍펜션,NaN,1
940,ji_yeong7996,2015-09-20 00:00:00+00:00,6,간절곶 우체통 가고 싶다고 늘 얘기 했는데 이제 왔엉!! 다음엔 오늘 멍때리다 못 ...,강원도 속초 듀플렉스 뷰펜션 스파펜션 바베큐 소고기 가라비 존맛 휴가 일상,NaN,1
941,k___k___s,2015-09-19 00:00:00+00:00,49,지금처럼..☺️ 301주,두번째가족여행 배부름 즐거움 해피해피함 아이코내무릎 두상구봉이도함께 태안 뷰펜션 1...,NaN,1


# sukso + sukso_reco : MERGE

In [18]:
# sukso_recommendation = pd.read_csv('숙소추천.csv')
# sukso = pd.read_csv('숙소.csv')

In [19]:
sukso = sukso[sukso['hashtag'].str.contains('숙소추천')==0]

In [20]:
sukso_merge = pd.concat([sukso_recommendation,sukso] , axis= 0)

In [21]:
index_del2 = sukso_merge[sukso_merge['hashtag'] == '[]'].index
sukso_merge = sukso_merge.drop(index_del2)

In [22]:
sukso_merge_list = [] # 비어있는 리스트 만들기 
for txt in sukso_merge['hashtag'] : 
    del_br = txt.strip("[]")
    del_br = del_br.replace("#"," ")
    del_br = del_br.replace(",","")
    del_br = del_br.replace("'","")
    sukso_merge_list.append(del_br) #값 채우기

In [23]:
sukso_merge_series = pd.Series(sukso_merge_list)

In [24]:
sukso_merge_series= sukso_merge_series.str.contains("뷰펜션") ==0
sukso_merge_series= sukso_merge_series[sukso_merge_series] 

In [25]:
sukso_merge.reset_index(drop = True, inplace = True)

In [26]:
sukso_merge['hashtag'] =sukso_merge_list

In [27]:
for i in range(len(sukso_merge['hashtag'])):
    if(len(sukso_merge['hashtag'].iloc[i])<=2):
        sukso_merge['hashtag'].iloc[i] = rmEmoji(sukso_merge['hashtag'].iloc[i])

C:\Users\morni\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [28]:
index_del = sukso_merge[sukso_merge['hashtag'] == ''].index
sukso_merge = sukso_merge.drop(index_del)

In [29]:
sukso_merge.reset_index(drop = True, inplace = True)

In [30]:
sukso_merge

,id,date,like,text,hashtag,src,target
0,busansomang,2021-06-21 00:00:00+00:00,"2,086","감성숙소 모음리스트✈️ 갬성, 힐링 모든 면에서 대박인 제쥬도 존예 숙소 다 모아와...",제주 제주 제주도 숙소 제주숙소 숙소추천 제주숙소추천 제주여행 제주...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
1,zzazang_,2021-06-26 00:00:00+00:00,"3,385",이거 하나보고 예약했던 숙소 🪨🪵🐷🥩🥬🍹💖 . . . 2일,제주여행 제주도여행 제주도 제주 제주숙소 제주숙소추천 제주독채 제주독...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
2,ejiyang,2021-06-09 00:00:00+00:00,"1,785",문의주신 숙소 공유해드려요♥️ 서귀포에 위치한 이에요🌼 이번 제주는 서귀포쪽에 머물...,부영호텔,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
3,so__hyeon,2021-06-26 00:00:00+00:00,"14,145",여기 진짜 미쳤다. 너무너무 좋다는 말 밖에🏕🍃 우리 사이트 바로 앞으로 바람 솔솔...,캠핑 캠핑장추천 캠핑장 캠핑스타...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
4,stayfolio\n인증됨,2021-06-24 00:00:00+00:00,"1,892","인증됨 작고(釐毫) 소박한 즐거움(小樂)을 더하다, 화려하게 꾸며진 공간은 많다. ...",이호소락 스테이폴리오 스테이폴리오_서울 북촌 노천탕,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
...,...,...,...,...,...,...,...
7122,mhc022012,2021-03-30 00:00:00+00:00,영상,Happy pottery studio Jeju 행복공방 에치비 도자기체험 제주 Ma...,체험 가볼만한곳 제주 서귀포 도자기 도예 귤 감귤 중문 숙소 카...,NaN,0
7123,seo_yeong_star,2021-03-30 00:00:00+00:00,246,방문자리뷰에 올려주신 후기 사진이예요 파란하늘과 마루방을 너무 이쁘게 담아주셔서 가...,누마루 파란하늘 방문자리뷰 스테이황룡 한옥스타그램 한옥스테이 한옥게스트...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
7124,___ju1004,2021-03-30 00:00:00+00:00,24,제주도에서의 하루^^ 제주도도 황사가 장난아님...;;; . . . 13주,정방폭포 제주도 황사 신행 신혼여행 숙소,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
7125,hyejini_vv,2021-03-30 00:00:00+00:00,20,제주 여행 여덟번째기록📝 노잼여행을 유잼여행으로 만들어주신 @youngmi__s2 ...,마지막 기록 제주 여행 뚜벅이 혼여행 마지막피드 즐거웠다 날씨는 ...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0


# 머신러닝 학습

In [31]:
sukso_merge_data= sukso_merge.iloc[:, [4,6]]
hanok_data = hanok_df.iloc[:, [4,6]]

In [32]:
hanok_plus_sukso_merge = pd.concat([hanok_data,sukso_merge_data], axis = 0)

In [33]:
hanok_plus_sukso_merge_X = hanok_plus_sukso_merge.iloc[:, 0]
hanok_plus_sukso_merge_y = hanok_plus_sukso_merge.iloc[:, 1]

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(hanok_plus_sukso_merge_X, # 문제데이터
                                                    hanok_plus_sukso_merge_y,  #정답데이터
                                                   random_state = 3,  # 섞는 방법
                                                   test_size =0.3)

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
testCV = CountVectorizer()

In [37]:
testCV.fit(X_train, y_train)

CountVectorizer()

In [38]:
X_train_CV = testCV.transform(X_train)
X_test_CV = testCV.transform(X_test)

In [39]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
svm = LinearSVC()

In [40]:
svm_result = cross_val_score(svm, X_train_CV, y_train, cv=5)
svm_result.mean()

0.9952204550977057

In [41]:
svm.fit(X_train_CV,y_train)

LinearSVC()

# 숙소_숙소추천에서 특정 카테고리 분류하기

In [42]:
sukso_recommendation = pd.read_csv('숙소추천.csv')
sukso = pd.read_csv('숙소.csv')

In [43]:
sukso = sukso[sukso['hashtag'].str.contains('숙소추천')==0]

In [44]:
sukso_merge = pd.concat([sukso_recommendation,sukso] , axis= 0)

In [45]:
index_del2 = sukso_merge[sukso_merge['hashtag'] == '[]'].index
sukso_merge = sukso_merge.drop(index_del2)

In [46]:
sukso_merge_list = [] # 비어있는 리스트 만들기 
for txt in sukso_merge['hashtag'] : 
    del_br = txt.strip("[]")
    del_br = del_br.replace("#"," ")
    del_br = del_br.replace(",","")
    del_br = del_br.replace("'","")
    sukso_merge_list.append(del_br) #값 채우기

In [47]:
sukso_merge_series = pd.Series(sukso_merge_list)

In [48]:
sukso_merge.reset_index(drop = True, inplace = True)

In [49]:
sukso_merge['hashtag'] =sukso_merge_list

In [50]:
X_train_CV = testCV.transform(sukso_merge.iloc[:, 4])

In [51]:
predict_array = svm.predict(X_train_CV)

In [52]:
one_numpy = np.where(predict_array==1)

In [53]:
one_numpy

(array([1023], dtype=int64),)

In [54]:
one_list = list(one_numpy)

In [55]:
one_series = pd.Series(one_list)

In [56]:
one_list = one_list[0].tolist()

In [57]:
predict_df = sukso_merge.iloc[(one_list)]

In [58]:
index_del = predict_df[predict_df['like'] == '영상'].index
predict_df = predict_df.drop(index_del)

In [59]:
predict_df.keys()

Index(['id', 'date', 'like', 'text', 'hashtag', 'src', 'target'], dtype='object')

In [60]:
predict_df.reset_index(drop = True, inplace = True)

In [61]:
for i in range(len(predict_df)):
    predict_df['like'].iloc[i] = predict_df['like'].iloc[i].replace(",","")
    predict_df['like'].iloc[i] = int(predict_df['like'].iloc[i])

C:\Users\morni\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [62]:
predict_df = predict_df.sort_values(by = ['like'], axis = 0, ascending = False)

In [66]:
predict_df

,id,date,like,text,hashtag,src,target
0,forestpension1,2021-06-08 00:00:00+00:00,15,숲속향기펜션 2층 공간 사진이예요🙂 테라스도 있어서 뷰가 아주 좋은곳랍니다 ㅎㅎ🌳 ...,강원도여행 강원도숙소 강원도펜션 펜션예약 양양군 강원도양양 강원도양양여...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0


In [64]:
predict_df.to_csv("뷰펜션_정답목록.csv", encoding="utf-8-sig")

In [65]:
print("학습시킨 데이터의 양 : ", len(hanok))

학습시킨 데이터의 양 :  1050
